## Unit 4 | Assignment - Pandas, Pandas, Pandas

## Background

The data dive continues!

## Option 1: Heroes of Pymoli

![Fantasy](Images/Fantasy.jpg)

Congratulations! After a lot of hard work in the data munging mines, you've landed a job as Lead Analyst for an independent gaming company. You've been assigned the task of analyzing the data for their most recent fantasy game Heroes of Pymoli. 

Like many others in its genre, the game is free-to-play, but players are encouraged to purchase optional items that enhance their playing experience. As a first task, the company would like you to generate a report that breaks down the game's purchasing data into meaningful insights.

In [1164]:
!pwd

/Users/pooya/Documents/DataScienceCource/homeworks/04-pandas


In [1165]:
import os
import json
import pandas as pd
import numpy as np

In [1166]:
#path = "./HeroesOfPymoli/purchase_data.json"
path = "./HeroesOfPymoli/purchase_data2.json"

In [1167]:
def topercent(num): return "{0:.2f}%".format(num*100)

In [1168]:
with open(path, 'r') as file:
    purchase_data = pd.DataFrame(json.load(file))
purchase_data.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,20,Male,93,Apocalyptic Battlescythe,4.49,Iloni35
1,21,Male,12,Dawne,3.36,Aidaira26
2,17,Male,5,Putrid Fan,2.63,Irim47
3,17,Male,123,Twilight's Carver,2.55,Irith83
4,22,Male,154,Feral Katana,4.11,Philodil43


Your final report should include each of the following:

**Player Count**

* Total Number of Players

In [1169]:
players_count = purchase_data["SN"].unique().size
pd.DataFrame({"Total Number of Players" : [players_count]})

,Total Number of Players
0,74


**Purchasing Analysis (Total)**

* Number of Unique Items
* Average Purchase Price
* Total Number of Purchases
* Total Revenue

In [1180]:
pd.DataFrame(
    {
        "Number of Unique Items" : [purchase_data["Item ID"].unique().size],
        "Average Purchase Price" : [purchase_data["Price"].mean()],
        "Total Number of Purchases" : [purchase_data["SN"].size],
        "Total Revenue" : [purchase_data["Price"].sum()]
    }
)

,Average Purchase Price,Number of Unique Items,Total Number of Purchases,Total Revenue
0,2.924359,64,78,228.1


**Gender Demographics**

* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [1181]:
# this is to remove duplicated players for cases when a player made multiple purchses
gender_stats = purchase_data[["Gender", "SN"]].groupby(["Gender","SN"]).count()
gender_stats["Count"] = 1
gender_stats = gender_stats.groupby(["Gender"]).count()
gender_stats["Count %"] = gender_stats["Count"] / gender_stats["Count"].sum() * 100
gender_stats

,Count,Count %
Gender,,
Female,13,17.567568
Male,60,81.081081
Other / Non-Disclosed,1,1.351351


**Purchasing Analysis (Gender)** 

* The below each broken by gender
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Normalized Totals

In [1172]:
gender_purchase_analysis = purchase_data[["Gender", "Price"]].groupby(["Gender"]).agg(["count", "mean", "sum"])
gender_purchase_analysis.columns = ["Purchase Count", "Average Purchase Price", "Total Purchase Value"]
gender_purchase_analysis["Normalized Totals"] = gender_purchase_analysis["Total Purchase Value"] / gender_purchase_analysis["Total Purchase Value"].sum()
gender_purchase_analysis.reset_index()

,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
0,Female,13,3.183077,41.38,0.181412
1,Male,64,2.884375,184.60,0.809294
2,Other / Non-Disclosed,1,2.120000,2.12,0.009294


**Age Demographics**

* The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.) 
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Normalized Totals

In [1173]:
age_bins = [0, 10, 14, 19, 200]
age_bins_names = ["<10", "10-14", "15-19", ">20"]
ages = np.array(purchase_data["Age"],dtype="int")
purchase_data["Age Tier"] = pd.cut(ages, age_bins, labels= age_bins_names)#
purchase_data.head()

,Age,Gender,Item ID,Item Name,Price,SN,Age Tier
0,20,Male,93,Apocalyptic Battlescythe,4.49,Iloni35,>20
1,21,Male,12,Dawne,3.36,Aidaira26,>20
2,17,Male,5,Putrid Fan,2.63,Irim47,15-19
3,17,Male,123,Twilight's Carver,2.55,Irith83,15-19
4,22,Male,154,Feral Katana,4.11,Philodil43,>20


In [1174]:
age_groups = purchase_data[["Age Tier", "SN", "Price"]].groupby(["Age Tier"])
age_groups_stats = age_groups.agg({"Price" : ["count", "mean", "sum"]})
age_groups_stats["Normalized Totals"] = age_groups_stats[("Price","sum")] / total_revenue
age_groups_stats = age_groups_stats.reset_index(level=None)
age_groups_stats.columns = ["Age Range", "Purchase Count", "Average Spent", "Total Spent", "Normalized Totals"]
age_groups_stats

,Age Range,Purchase Count,Average Spent,Total Spent,Normalized Totals
0,<10,5,2.764000,13.82,0.060587
1,10-14,3,2.986667,8.96,0.039281
2,15-19,11,2.764545,30.41,0.133319
3,>20,59,2.964576,174.91,0.766813


**Top Spenders**

* Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
  * SN
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value

In [1175]:
spent_total = purchase_data[["SN","Price"]].groupby(["SN"])
spent_total = spent_total.agg({"Price":["count", "mean", "sum"]})
spent_total = spent_total.rename(columns={"Price": "Total Purchase Value"})
spent_total = spent_total.reset_index()
spent_total = spent_total.sort_values(by=[("Total Purchase Value", "sum")],ascending=False)[:5]
spent_total

SN Total Purchase Value             
                              count   mean   sum
63   Sundaky74                    2  3.705  7.41
4    Aidaira26                    2  2.565  5.13
23     Eusty71                    1  4.810  4.81
16  Chanirra64                    1  4.780  4.78
8     Alarap40                    1  4.710  4.71

**Most Popular Items**

* Identify the 5 most popular items by purchase count, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

In [1176]:
purchase_data.head()

,Age,Gender,Item ID,Item Name,Price,SN,Age Tier
0,20,Male,93,Apocalyptic Battlescythe,4.49,Iloni35,>20
1,21,Male,12,Dawne,3.36,Aidaira26,>20
2,17,Male,5,Putrid Fan,2.63,Irim47,15-19
3,17,Male,123,Twilight's Carver,2.55,Irith83,15-19
4,22,Male,154,Feral Katana,4.11,Philodil43,>20


In [1177]:
most_popular = purchase_data[['Item ID', 'Item Name', 'Price']].groupby(['Item ID', 'Item Name', 'Price'])
most_popular = most_popular.agg({"Price":["sum", "count"]})
most_popular = most_popular.rename(columns={"sum":"Total Purchase Value", "count" : "Purchase Count"})
most_popular.sort_values(by=[("Price", "Purchase Count")], ascending = False)[:5].reset_index()

Item ID       Item Name Price                                    
                                Total Purchase Value Purchase Count
0      94  Mourning Blade  3.64                10.92              3
1      90        Betrayer  4.12                 8.24              2
2     111    Misery's End  1.79                 3.58              2
3      64   Fusion Pummel  2.42                 4.84              2
4     154    Feral Katana  4.11                 8.22              2

**Most Profitable Items**

* Identify the 5 most profitable items by total purchase value, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

In [1178]:
most_popular.sort_values(by=[("Price", "Total Purchase Value")], ascending = False)[:5].reset_index()

Item ID                          Item Name Price                       \
                                                   Total Purchase Value   
0      94                     Mourning Blade  3.64                10.92   
1     117  Heartstriker, Legacy of the Light  4.71                 9.42   
2      93           Apocalyptic Battlescythe  4.49                 8.98   
3      90                           Betrayer  4.12                 8.24   
4     154                       Feral Katana  4.11                 8.22   

                  
  Purchase Count  
0              3  
1              2  
2              2  
3              2  
4              2

As final considerations:

* Your script must work for both data-sets given.
* You must use the Pandas Library and the Jupyter Notebook.
* You must submit a link to your Jupyter Notebook with the viewable Data Frames. 
* You must include an exported markdown version of your Notebook called  `README.md` in your GitHub repository.  
* You must include a written description of three observable trends based on the data. 
* See [Example Solution](HeroesOfPymoli/HeroesOfPymoli_Example.pdf) for a reference on expected format. 